### Documentación

Autor:  Julian David Fernandez
Curso:  Data Engineer - Coderhouse
Fecha:  6 Abril - 2023
Ver  : 1.0.0

#### Proposito

Creación de una ETL, para extracción de informaciíon climatica (ultimos 8 dias) de todos los aeropuertos comerciales del estado de la Floria en los Estados unidos. La informaciíon geografica se encuentra complementada con la información geografica del aeropuerto.

URL API: https://docs.aviationapi.com/

In [9]:
# Importar las librerias
import pandas as pd
import numpy as np
import sqlalchemy
import requests

### Paso 1 - EXTRACT

Listado de aeropuertos a consultar

In [10]:
# inicializar lista de lista con los nombres de los aeropuertos de la florida y los codigos FAA y ICAO
data = [['Daytona Beach',	'DAB',	'KDAB'],['Fort Lauderdale',	'FLL',	'KFLL'],['Fort Myers',	'RSW',	'KRSW'],['Fort Walton Beach',	'VPS',	'KVPS'],['Gainesville',	'GNV',	'KGNV'],['Jacksonville',	'JAX',	'KJAX'],['Key West',	'EYW',	'KEYW'],['Melbourne',	'MLB',	'KMLB'],['Miami',	'MIA',	'KMIA'],['Orlando',	'MCO',	'KMCO'],['Panama City',	'ECP',	'KECP'],['Pensacola',	'PNS',	'KPNS'],['Punta Gorda',	'PGD',	'KPGD'],['Sanford',	'SFB',	'KSFB'],['Sarasota',	'SRQ',	'KSRQ'],['St. Petersburg',	'PIE',	'KPIE'],['Tallahassee',	'TLH',	'KTLH'],['Tampa',	'TPA',	'KTPA'], ['West Palm Beach',	'PBI',	'KPBI']]

# Creacion del dataframe 
df_airports = pd.DataFrame(data, columns=['Name', 'FAA', 'ICAO'])

In [11]:
# Extraccion de datos geograficos: API
get_params = '?apt=' + ",".join(df_airports.FAA.values.tolist())
url_endpoint_geo = 'https://api.aviationapi.com/v1/airports' + get_params
url_endpoint_geo

'https://api.aviationapi.com/v1/airports?apt=DAB,FLL,RSW,VPS,GNV,JAX,EYW,MLB,MIA,MCO,ECP,PNS,PGD,SFB,SRQ,PIE,TLH,TPA,PBI'

In [12]:
# Extraccion de datos climaticos: API
url_endpoint_weather = 'https://api.aviationapi.com/v1/weather/metar' + get_params
url_endpoint_weather

'https://api.aviationapi.com/v1/weather/metar?apt=DAB,FLL,RSW,VPS,GNV,JAX,EYW,MLB,MIA,MCO,ECP,PNS,PGD,SFB,SRQ,PIE,TLH,TPA,PBI'

### Paso 2 - TRANSFORM

Extraccion - Consulta de información geografica de los aeropuertos

In [13]:
# Consulta de información geografica por medio de un Request HTTP
headers = {'Accept': 'application/json'}
r_geo = requests.get(url_endpoint_geo, headers=headers)

# Conversión de json a objeto en python 
airport_geo_dict = r_geo.json()

# y luego transformar los diccionarios en DataFrames
temp = []

for k, vals in airport_geo_dict.items():
    temp.append(vals[0])

df_airport_geo = pd.DataFrame(temp)

In [30]:
df_airport_geo.head()

,site_number,type,facility_name,faa_ident,icao_ident,region,district_office,state,state_full,county,...,certification_typedate,customs_airport_of_entry,military_joint_use,military_landing,lighting_schedule,beacon_schedule,control_tower,unicom,ctaf,effective_date
0,03147.*A,AIRPORT,DAYTONA BEACH INTL,DAB,KDAB,ASO,ORL,FL,FLORIDA,VOLUSIA,...,I C S 05/1973,N,N,Y,SS-SR,SS-SR,Y,122.95,,11/04/2021
1,03192.*A,AIRPORT,FORT LAUDERDALE/HOLLYWOOD INTL,FLL,KFLL,ASO,ORL,FL,FLORIDA,BROWARD,...,I E S 05/1973,Y,N,Y,SS-SR,SS-SR,Y,122.95,,11/04/2021
2,03198.2*A,AIRPORT,SOUTHWEST FLORIDA INTL,RSW,KRSW,ASO,ORL,FL,FLORIDA,LEE,...,I D S 05/1983,N,,Y,SEE RMK,SS-SR,Y,122.95,128.75,11/04/2021
3,03534.*A,AIRPORT,EGLIN AFB/DESTIN-FT WALTON BEACH,VPS,KVPS,ASO,ORL,FL,FLORIDA,OKALOOSA,...,I C S 05/1973,N,Y,N,SS-SR,SS-SR,Y,,,11/04/2021
4,03210.*A,AIRPORT,GAINESVILLE RGNL,GNV,KGNV,ASO,ORL,FL,FLORIDA,ALACHUA,...,I B S 05/1973,N,N,Y,SEE RMK,SS-SR,Y,122.95,119.55,11/04/2021


In [14]:
# Consulta de información climatica por medio de un Request HTTP
r_weather = requests.get(url_endpoint_weather, headers=headers)

# Conversión de json a objeto en python 
airport_weather_dict = r_weather.json()
airport_weather_dict
# y luego transformar los diccionarios en DataFrames
temp = []

# ajuste del objeto json con el objetivo de transformar todos los niveles del objeto en una tabla columnar.
for k, vals in airport_weather_dict.items():

    sky_conditions = vals['sky_conditions'][0]
    sky_conditions["sky_conditions.coverage"] = sky_conditions.pop("coverage")
    sky_conditions["sky_conditions.base_agl"] = sky_conditions.pop("base_agl")
    vals.update(sky_conditions)
    temp.append(vals)

df_airport_weather = pd.DataFrame(temp)

# limpieza de información raw (datos crudos) la cual es redundante para el dataset.
del df_airport_weather['raw']
del df_airport_weather['sky_conditions']


In [31]:
df_airport_weather.head()

,station_id,temp,dewpoint,wind,wind_vel,visibility,alt_hg,alt_mb,wx,auto_report,category,report_type,time_of_obs,sky_conditions.coverage,sky_conditions.base_agl
0,KJAX,28.9,19.4,110,7,10.0,30.15,1021.5,None,true,VFR,METAR,2023-04-06T19:56:00Z,SCT,3600
1,KVPS,28.1,17.3,170,8,10.0,30.15,1021.9,None,true,VFR,SPECI,2023-04-06T19:55:00Z,FEW,2500
2,KMLB,28.3,18.9,100,14,10.0,30.15,1021.3,None,true,VFR,METAR,2023-04-06T19:53:00Z,CLR,None
3,KPBI,29.4,19.4,100,11,10.0,30.15,1021.3,None,true,VFR,METAR,2023-04-06T19:53:00Z,SCT,3500
4,KPGD,31.7,16.1,340,7,10.0,30.12,1019.8,None,true,VFR,METAR,2023-04-06T19:53:00Z,BKN,7000


### Pase 3 - LOAD

Conjunto de pasos para la carga de datos dentro de Amazon Redshift, carga de la informacion geografica y la información climatica.

In [32]:
# creacion del engine para la carga de datos directo a AWS Redshift
url       = "xxxxxxxx.us-east-1.redshift.amazonaws.com" 
port      = "5439"
data_base = "data-engineer-database" 
user      = "XXXXXXXX_coderhouse"
pwd       = "xxxxxxxx"
myschema  = "xxxxxxxx_coderhouse" 


In [17]:
from sqlalchemy import create_engine

#Se crear la conexión
conn = create_engine(f'postgresql://{user}:{pwd}@{url}.com:{port}/{data_base}')

In [33]:
# Carga de la información actualizada del clima, se utiliza la opcion append, para tener un historico del clima, cada vez que se ejecuta
# el ETl.
df_airport_weather.to_sql('proyecto_weather', conn, index=False, if_exists='append', schema=myschema)

In [ ]:
# Carga de la información actualizada del aeropuerto, se remplaza la información de los aeropeurtos, por no ser una varible de tiempo
# sino que contiene información descriptiva y de estado de cada aeropuerto.
df_airport_geo.to_sql('proyecto_airports', conn, index=False, if_exists='replace', schema=myschema)